In [51]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers import default_data_collator
from transformers import Wav2Vec2FeatureExtractor, HubertForSequenceClassification
import torch
import torch.nn as nn
from datasets import Dataset, Audio, Value, Features,load_dataset,ClassLabel
from transformers import Wav2Vec2Processor
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
from transformers import AdamW,get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm
import os
from torch.utils.data import DataLoader
from datasets import load_metric


def featurize(batch):
#     audio_arrays = [batch['audio'][i]['array'] for i in range(len(batch))]
    audio_arrays = [batch['audio'][i]['array'] for i in range(len(batch['id']))]
#     print(len(audio_arrays))
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=16_000, 
        max_length=int(16_000 * 10),  # 10s
        truncation=True, 
        padding='max_length',
    )
    return inputs


checkpoint = "facebook/wav2vec2-base"
weighted_sum = False
# checkpoint = "facebook/hubert-base-ls960"
# checkpoint = "facebook/wav2vec2-large-lv60"

x = [str(i) for i in range(0,100,1)]
features = Features(
    {
        "id": Value("string"),
        "speaker_id": Value("string"), 
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000),
        "label": ClassLabel(num_classes=100,names=x,names_file=None,id=None)
    }
)


dataset = load_dataset('csv', 
                       data_files={'test': '../data/identification/test_100.csv'},
                       features=features)
dataset = dataset.map(remove_columns=(['path','speaker_id']),num_proc=24)
dataset = dataset.sort("label")
# sampling_rate = dataset.features["audio"].sampling_rate
if 'base' in checkpoint and not weighted_sum:
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(checkpoint,return_attention_mask=False)
else:
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(checkpoint,return_attention_mask=True)


dataset = dataset.map(featurize, remove_columns='audio',batched=True,num_proc=20,batch_size=1)

# dataset = dataset.rename_column('id','label_ids')

if 'base' in checkpoint and not weighted_sum:
    dataset.set_format("torch",columns=["id","input_values", "label"])
else:
    dataset.set_format("torch",columns=["id","input_values", "attention_mask", "label"])

Using custom data configuration default-43c2fe400d73e135
Found cached dataset csv (/storage/home/hcocice1/vkotra3/.cache/huggingface/datasets/csv/default-43c2fe400d73e135/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 389.08it/s]
Loading cached processed dataset at /storage/home/hcocice1/vkotra3/.cache/huggingface/datasets/csv/default-43c2fe400d73e135/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-2b43112609dacf4d.arrow
Loading cached processed dataset at /storage/home/hcocice1/vkotra3/.cache/huggingface/datasets/csv/default-43c2fe400d73e135/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ecfe8e2cea57772c.arrow
Loading cached processed dataset at /storage/home/hcocice1/vkotra3/.cache/huggingface/datasets/csv/default-43c2fe400d73e135/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-07049e0558fd57b1.arrow
Loading cached processed dataset at /s

#8:   3%|▎         | 1/32 [00:01<00:54,  1.76s/ba]

















#18:   3%|▎         | 1/31 [00:01<00:48,  1.61s/ba]










#11:   3%|▎         | 1/32 [00:01<00:53,  1.71s/ba]


#3:   3%|▎         | 1/32 [00:01<00:57,  1.84s/ba]



#4:   3%|▎         | 1/32 [00:01<00:56,  1.82s/ba]













#14:   3%|▎         | 1/31 [00:01<00:50,  1.67s/ba]








#9:   3%|▎         | 1/32 [00:01<00:54,  1.76s/ba]






#7:   3%|▎         | 1/32 [00:01<00:55,  1.79s/ba]














#15:   3%|▎         | 1/31 [00:01<00:49,  1.66s/ba]
















#17:   3%|▎         | 1/31 [00:01<00:49,  1.65s/ba]


















 ... (more hidden) ...












#13:  61%|██████▏   | 19/31 [00:01<00:00, 14.70ba/s]




#0:  56%|█████▋    | 18/32 [00:01<00:01, 12.52ba/s]









#10:  50%|█████     | 16/32 [00:01<00:01, 12.01ba/s]















#16:  52%|█████▏    | 16/31 [00:01<00:01, 12.67ba/s]

#1:  56%|█████▋    | 18/32 [00:01<00:01, 12.56ba/s]





#6:  50%|█████     | 16/32 [00:01<00:01, 11.58

### Model

In [52]:
class CustomBaseSID(nn.Module):
    def __init__(self,checkpoint,num_labels,inter_layer_num,attend):
        
        ### attend is a boolean
        super(CustomBaseSID, self).__init__()
#         self.hubert = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-sid")
        self.model =AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=attend,output_hidden_states=True))
        self.num_labels = num_labels
        self.attend_mask = attend
        ### New layers:
        self.linear1 = nn.Linear(768, 1024)
        self.linear2 = nn.Linear(1024, num_labels)
        
        ### Intermediate Layer Number
        self.layer_num = inter_layer_num

    def forward(self, input_values=None, attention_mask=None,labels=None):
        if(self.attend_mask):
            outputs = self.model(input_values=input_values, attention_mask=attention_mask)
        else:
            outputs = self.model(input_values=input_values, attention_mask=None)
        feature = outputs.hidden_states[self.layer_num-1]
        agg_vec_list = []
        for i in range(len(feature)):
            if(attention_mask==None):
                length = len(feature[i])
            
            else:
                if torch.nonzero(attention_mask[i] < 0, as_tuple=False).size(0) == 0:
                    length = len(feature[i])
                else:
                    length = torch.nonzero(attention_mask[i] < 0, as_tuple=False)[0] + 1
            agg_vec=torch.mean(feature[i][:length], dim=0)
            agg_vec_list.append(agg_vec)
        mean = torch.stack(agg_vec_list)
        # sequence_output has the following shape: (batch_size, sequence_length, 768)
        linear1_output = self.linear1(mean) ## extract the 1st token's embeddings
        logits = self.linear2(linear1_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)


In [53]:
def freeze_layers_transformer(model_ft,keywords,inter_layer,encoder_mode=True):
    ct = 0
    for child in model_ft.children():
        for name,param in child.named_parameters():
            if(encoder_mode):
                if 'encoder' in name and str(ct) in name:
                    ct += 1
                    if ct < inter_layer:
                        param.requires_grad = False
                    
            for word in keywords:         
                if word in name:
                    param.requires_grad = False
                            
    return model_ft

In [54]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'label', 'input_values'],
        num_rows: 632
    })
})

In [55]:
inter_layer = 12
dirpath="/storage/home/hcocice1/vkotra3/6254_Project/code/w2v-base/"+str(inter_layer)+'/'
PATH = dirpath+str(4)+'.pt'
checkpoint = "facebook/wav2vec2-base"

if 'base' in checkpoint:
    attend = False

device = "cpu"
model_ft = CustomBaseSID(checkpoint=checkpoint,num_labels=100,inter_layer_num=inter_layer,attend=attend)
model_ft.load_state_dict(torch.load(PATH, map_location=device))

metric_name="accuracy"
metric = load_metric(metric_name)
test_dataloader = DataLoader(dataset["test"], batch_size=1, collate_fn=default_data_collator)


/storage/home/hcocice1/vkotra3/.local/lib/python3.9/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint 

In [82]:
# keys = list(dataset['test'].features.keys())
keys = list(dataset['test']['id'])
test_dict = dict.fromkeys(keys)

In [88]:
list(test_dict.keys())[0]

'id10001-Y8hIVOBuels-00001'

In [90]:
def evaluate(test_dataloader,model_ft):
    test_dict = dict.fromkeys(list(dataset['test']['id']))
    keys = list(test_dict.keys())
    
    for i,batch in enumerate(test_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model_ft(**batch)
        logits = outputs.logits
        sm = torch.nn.Softmax(dim=1)
        probabilities = sm(logits)
        test_dict[keys[i]] = dict.fromkeys(['labels','probabilities','predictions'])
        test_dict[keys[i]]['labels'] = batch['labels'].detach().numpy()
        test_dict[keys[i]]['probabilities'] = probabilities[0].detach().numpy()
        predictions = torch.argmax(logits, dim=-1)
        test_dict[keys[i]]['predictions'] = predictions[0].detach().numpy()
        metric.add_batch(predictions=predictions, references=batch["labels"])
    accuracy = metric.compute()
    return test_dict,accuracy

In [94]:
w2vbase_12,w2vbase_12_acuraccy = evaluate(test_dataloader,model_ft)


KeyboardInterrupt



In [96]:
w2vbase_12

{'id10001-Y8hIVOBuels-00001': {'labels': array([0]),
  'probabilities': array([3.94152731e-01, 4.84397387e-05, 2.61283276e-04, 6.72649359e-04,
         1.38085503e-02, 3.24054068e-04, 7.41353797e-05, 1.01796595e-05,
         4.01328392e-02, 1.19099033e-03, 5.48485434e-04, 1.22963211e-05,
         2.25878815e-04, 1.65788751e-05, 6.58128876e-04, 2.21315640e-05,
         1.69755749e-05, 2.46690935e-04, 5.94912744e-05, 1.30179594e-03,
         7.17953313e-04, 2.49014884e-05, 2.79265252e-04, 1.18639227e-03,
         1.05882809e-05, 3.46931280e-04, 1.56726374e-03, 5.90060772e-05,
         4.04923747e-04, 1.03173646e-04, 5.08171564e-04, 3.59937512e-05,
         4.55926667e-04, 4.30798445e-06, 2.22485699e-03, 9.14659977e-05,
         5.86151902e-04, 1.89333980e-04, 2.28658228e-05, 9.02370266e-06,
         4.56147623e-04, 3.44344880e-04, 4.00235222e-05, 5.61896813e-05,
         3.82361904e-05, 2.51190140e-05, 5.35446234e-05, 2.91154836e-04,
         5.84921145e-06, 1.06375774e-04, 4.15303948e-0